In [1]:
#### GIS PROGRAMMING TO FIND A LOCATION FOR A NEW ASTRONOMICAL OBSERVATORY IN FRANCE ####
#### Sannier Maelle ####
#### 04.30.2020 ####
#### EPF Graduate School of Engineering, Sceaux, France - RUDN, Moscow, Russia ####
#### Instructor: Naci Dilekli

######## Parameters ######### 
# workspace = storage space defined by the user e.g "C:/Observatories"
# fc1 = first layer to use to create buffers : french cities
# b1  = Radius Buffer in Kilometers for 1rst feature class
# fc2 = second layer to use to create buffers : existing observatories
# b2  = Radius buffer in kilometers for 2nd feature class
# citysize = We want to avoid cities over citysize inhabitants
# coef1 = coefficient in the weighted sum for Crit1
# coef2 = coefficient in the weighted sum for Crit2
# coef3 = coefficient in the weighted sum for Crit3
# percentage = percentage of best pixels of the weithed sum layer that the user wants to keep

def Find_Observatories(workspace,b1,b2,citysize,coef1,coef2,coef3,percentage):
    
######## Initialization ######### 
        from datetime import datetime
        start_time = datetime.now()
        
        import arcpy
        arcpy.env.workspace = workspace
        arcpy.env.overwriteOutput = True

        intermediate_layers=["fc1_buffer","fc2_buffer","Buffers_Union","Buffers_Dissolved",\
                             "Lum_Vector","Crit1VctEra","Crit1Rst","Crit1NormInv","Crit1Norm",\
                            "RelcassAlt_Vct","Crit2VctEra","Crit2Rst","Crit2Norm",\
                            "Clouds_Vector","Crit3VctEra","Crit3Rst","Crit3Norm",\
                            "arithmeticlayer","Bestpixels","Finalcities","BestpixelsVct","FinalResult"]
                             
        for intermediate_layer in intermediate_layers:
            if arcpy.Exists(intermediate_layer):
                arcpy.Delete_management(intermediate_layer)
                
######## Verification #########     
        
        if percentage > 100 or percentage <0 :
            print("The percentage must be between 0 and 100")
        
                
######## Buffers layers #########         
                
        #Creation of 2 buffers analysis around the cities over <citysize> inhabitants 
        #and existing astronomical observatories
            
        outselection = arcpy.SelectLayerByAttribute_management(workspace + "/French_cities/frenchville.shp",'NEW SELECTION', "PTOT >" + str(citysize)) 
        arcpy.Buffer_analysis(outselection, workspace +"/French_cities" + "/"+ "fc1_buffer.shp",b1)
        arcpy.Buffer_analysis(workspace+"/Observatories/Observatories.shp", workspace + "/Observatories" +"/"+ "fc2_buffer.shp",b2)
        print("First feature class: ",workspace + "/French_cities/frenchville.shp","\nSecond feature class: ",workspace+"/Observatories/Observatories.shp")
        print("Creating ",b1,"buffer around cities with more than",citysize,"inhabitants...","\nCreating  ",b2,"buffer around existing observatories...")        
        
        #Union and Dissolve of the 2 buffers analysis
        arcpy.Union_analysis([workspace +"/French_cities" + "/"+"fc1_buffer.shp",\
                              workspace + "/Observatories" +"/"+"fc2_buffer.shp"],"Buffers_Union")
        arcpy.Dissolve_management("Buffers_Union.shp","Buffers_Dissolved")
        
######## First Criteria : Luminous Pollution ########
        
        #Erase analysis with the Buffer Union layer
        #Finally convert it into a Raster
        arcpy.Erase_analysis(workspace + "/Luminous_Pollution" + "/" + "Lum_Vector.shp","Buffers_Dissolved.shp","Crit1VctEra")
        arcpy.FeatureToRaster_conversion("Crit1VctEra.shp","gridcode",workspace + "/Luminous_Pollution"+ "/" + "Crit1Rst",0.01)

        #Normalization of the layer into a 0-100 range
        luminouslayer = arcpy.Raster(workspace + "/Luminous_Pollution" + "/" + "Crit1Rst")
        luminousNormalized = (luminouslayer - luminouslayer.minimum) / (luminouslayer.maximum - luminouslayer.minimum) *100
        
        #Inversion of the layer so that values close to 100 represent the most wanted areas
        luminousInv = (luminousNormalized - luminousNormalized.maximum)* -1
        luminousInv.save(workspace + "/Luminous_Pollution" + "/" + "Crit1Norm.tif")
        
        print("Normalizing the luminous pollution layer...")
        
######## Second Criteria : Altitude ########
        
        #Erase analysis with the Buffer Union layer
        #Finally convert it into a Raster
        arcpy.Erase_analysis(workspace +"/Altitude" + "/"+ "RelcassAlt_Vct.shp","Buffers_Dissolved.shp","Crit2VctEra")
        arcpy.FeatureToRaster_conversion("Crit2VctEra.shp","gridcode",workspace + "/Altitude" +"/" + "Crit2Rst",0.01)
        
        #Normalization of the layer into a 0-100 range
        altitudelayer = arcpy.Raster(workspace + "/Altitude" +"/" + "Crit2Rst")
        altitudeNormalized = (altitudelayer - altitudelayer.minimum) / (altitudelayer.maximum-altitudelayer.minimum) *100
        altitudeNormalized.save(workspace + "/Altitude" +"/" + "Crit2Norm.tif")
        
        print("Normalizing the altitude layer...")
        
######## Third Criteria : Cloudiness ########
        
        #Erase analysis with the Buffer Union layer
        #Finally convert it into a Raster
        arcpy.Erase_analysis(workspace + "/Cloudiness" +"/"+ "Clouds_Vector.shp","Buffers_Dissolved.shp","Crit3VctEra")
        arcpy.FeatureToRaster_conversion("Crit3VctEra.shp","gridcode",workspace +"/Cloudiness" + "/" + "Crit3Rst",0.01)
        
        #Normalization of the layer into a 0-100 range
        cloudslayer = arcpy.Raster(workspace +"/Cloudiness" + "/" + "Crit3Rst")
        cloudsNormalized = (cloudslayer - cloudslayer.minimum) / (cloudslayer.maximum - cloudslayer.minimum) *100
        cloudsNormalized.save(workspace + "/Cloudiness" +"/" + "Crit3Norm.tif")

        print("Normalizing the cloudiness layer...")
        
######## Arithmeric Layer : weighted sum ########

        arithmeticlayer = coef1 * luminousInv + coef2 * altitudeNormalized + coef3 * cloudsNormalized
        arithmeticlayer.save(workspace + "/Results" +"/" + "arithmeticlayer.tif")
        
        print("Coefficients for weighted sum: ",\
              "\nCoeffient for lum pollution :\t",coef1,\
              "\nCoeffient for altitude :\t",coef2,\
              "\nCoeffient for cloudiness :\t",coef3)
        print("Creating the weighted sum layer... ")
        
######## Final Results ########
        
        #Definition of the percentage of best pixels the user want to keep
        #Then finding the minumum value of the pixels that the user want to keep
        condition = (arithmeticlayer.maximum - arithmeticlayer.minimum ) / (100/percentage)
        treshold = arithmeticlayer.maximum - condition
        print("Percentage of best pixels kept: ", percentage,"%")
                
        #Creation of the new layer of Best pixels
        outcon = arcpy.sa.Con(arithmeticlayer,1,"","Value >" + str(treshold))
        outcon.save(workspace  +"/Results"+"/" + "Bestpixels")
            
        #Final results giving a village in output 
        arcpy.RasterToPolygon_conversion(workspace  +"/Results"+"/" +"Bestpixels",workspace  +"/Results"+"/" +"BestpixelsVct.shp","NO_SIMPLIFY","VALUE")
        arcpy.Intersect_analysis([workspace  +"/Results"+"/" +"BestpixelsVct.shp",workspace  +"/French_cities"+"/" +"frenchville.shp"],workspace  +"/Results"+"/" +"FinalResult.shp")
        
        layerFields = ["*"]
        nbr_results = arcpy.GetCount_management(workspace + "/Results"+"/" + "FinalResult.shp")
        print("Nbr of results: ", nbr_results)
        with arcpy.da.SearchCursor(workspace + "/Results"+ "/" + "FinalResult.shp", layerFields) as cursor:
            print("\nCities:\n")
            for row in cursor:
                print("Name :",row[6],",", "GPS Coordinates: ",\
                      "( latitude:",row[11],"longitude:",row[12],")","\n")
                
        end_time = datetime.now()
        print('\nExecution time: {}'.format(end_time - start_time))

In [2]:
##EXAMPLE : 
Find_Observatories("C:/Observatories","30 Kilometers","30 Kilometers",50000,3,1,2,10) 

First feature class:  C:/Observatories/French_cities/frenchville.shp 
Second feature class:  C:/Observatories/Observatories/Observatories.shp
Creating  30 Kilometers buffer around cities with more than 50000 inhabitants... 
Creating   30 Kilometers buffer around existing observatories...
Normalizing the luminous pollution layer...
Normalizing the altitude layer...
Normalizing the cloudiness layer...
Coefficients for weighted sum:  
Coeffient for lum pollution :	 3 
Coeffient for altitude :	 1 
Coeffient for cloudiness :	 2
Creating the weighted sum layer... 
Percentage of best pixels kept:  10 %
Nbr of results:  14

Cities:

Name :  MŽlan , GPS Coordinates:  ( latitude: 44.2 longitude: 6.133333 ) 

Name :  Auribeau , GPS Coordinates:  ( latitude: 44.2 longitude: 6.166667 ) 

Name :  Ainac , GPS Coordinates:  ( latitude: 44.216667 longitude: 6.2 ) 

Name :  Lambert , GPS Coordinates:  ( latitude: 44.216667 longitude: 6.216667 ) 

Name :  Esclangon , GPS Coordinates:  ( latitude: 44.2166